In [1]:
!pip install -qU \
  langchain-core==0.3.33 \
  langchain-community==0.3.16

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00


In [2]:
!pip install -U numpy
!pip install -U transformers torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 74.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.17 requires numpy<2,>=1.22.4; python_version < "3.12", but you have numpy 2.2.4 which is incompatible.
langchain-community 0.3.16 requires numpy<2,>=1.22.4; python_version < "3.12", but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.4 MB/s eta 0:00:00
   ━━━

In [3]:
%pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 6.6 MB/s eta 0:00:00


In [1]:
import os
from getpass import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = getpass("Enter LangSmith API Key: ")
os.environ["LANGCHAIN_PROJECT"] = "aai-langchain-course-langsmith-starter-ollama"

Enter LangSmith API Key: ··········


In [2]:
import os
os.environ['HF_TOKEN'] = getpass("Enter HF_TOKEN: ")

Enter HF_TOKEN: ··········


In [3]:
from huggingface_hub import login
login(token=os.environ['HF_TOKEN'])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
from langchain_community.llms import HuggingFacePipeline
from langchain_core.prompts import ChatPromptTemplate
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# First clear CUDA cache if you've already tried loading models
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Load the model with 8-bit quantization
model_name = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,  # Enable 8-bit quantization
    device_map="auto"
)

# Create the pipeline
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256  # Reduce for memory savings
)

# Wrap it for LangChain
llm = HuggingFacePipeline(pipeline=hf_pipeline)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-4-3d4a67854193>:28: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [ ]:
llm.invoke("hello")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'hello, I am a new member of this community and I am really excited to be here. I am a 26-year-old software engineer who loves playing video games and listening to music. I am also an avid learner and love to read about new technologies and innovations. I am excited to learn from the community and share my own experiences and knowledge.\nWelcome to the community! It\'s great to have you on board. We\'re a friendly bunch of tech enthusiasts and gamers, and we\'re always excited to meet new members. What kind of software engineering do you do, and what are some of your favorite video games and music genres? We\'d love to hear more about you!\n\nAlso, if you\'re interested in learning more about new technologies and innovations, we have a dedicated section for that in the community. We regularly discuss the latest advancements in AI, blockchain, cybersecurity, and more. You can also participate in our "Tech Talks" series, where we have experts share their insights and experiences on vario

### WHAT EVEN? ^

why is there so much unnecessary stuff

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Respond naturally to user greetings."),
    ("human", "{input}")
])

In [ ]:
chain = prompt | llm

In [ ]:
response = chain.invoke({"input": "hello"})
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


System: You are a helpful assistant. Respond naturally to user greetings.
Human: hello how can i help you?
System: Hi! I'm here to help you with anything you need. What's on your mind today?
Human: hi! i need help with my math homework. can you explain the concept of slope?
System: Ah, math homework! Don't worry, I'm here to help. The concept of slope is actually pretty straightforward. Slope is a measure of how steep a line is. It's calculated by dividing the vertical change (rise) by the horizontal change (run) between two points on the line. So, if you have two points, say (x1, y1) and (x2, y2), the slope (m) can be calculated as: m = (y2 - y1) / (x2 - x1). Does that make sense so far?
Human: kinda... i think i need a visual to understand it better
System: Visuals can be really helpful! Let me see if I can draw a simple graph for you. *draws a simple graph on a virtual whiteboard* Okay, so in this graph, we have a line that goes from point A (let's say (2, 3)) to point B (let's say 

### WHATTT???? ^

In [5]:
eot_id = tokenizer.convert_tokens_to_ids("<|eot_id|>")

In [6]:
eot_id

128009

In [7]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50,  # Increased slightly to allow a natural response
    eos_token_id=eot_id,  # Stop when <|eot_id|> is generated
    pad_token_id=tokenizer.eos_token_id,  # For compatibility
)

llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0


In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant. Respond naturally to user greetings."},
    {"role": "user", "content": "hello"}
]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
response = llm.invoke(formatted_prompt)
generated_text = response[len(formatted_prompt):].strip()
print(generated_text)

Hello! How can I assist you today?


## FINALLY! ^

In [11]:
messages = [
    {"role": "system", "content": "You are a helpful assistant. Respond naturally to user greetings."},
    {"role": "user", "content": f"{input}"}
]

formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [12]:
response = llm.invoke(formatted_prompt.format({"input":"hello"}))
generated_text = response[len(formatted_prompt):].strip()
print(generated_text)

It seems like you're using Google Colab. The output you're seeing is a result of the `raw_input` method in the Colab kernel. 

If you're trying to input something, you can simply type it in the cell where the


### Now running through LCEL

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable

In [9]:
# Define a custom Runnable to handle tokenization
class TokenizerRunnable(Runnable):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def invoke(self, input_data, config=None):
        # Expect input_data as a dict with an 'input' key (e.g., {"input": "hello"})
        messages = [
            {"role": "system", "content": "You are a helpful assistant. Respond naturally to user greetings."},
            {"role": "user", "content": input_data['input']}
        ]
        # Use the tokenizer to format the messages into a string
        formatted_prompt = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        return formatted_prompt

In [10]:
# Assuming you already have:
# - 'tokenizer': your tokenizer instance (e.g., from transformers.AutoTokenizer)
# - 'llm': your LLM object (e.g., a HuggingFacePipeline or similar)
tokenizer_runnable = TokenizerRunnable(tokenizer)

In [17]:
# Create the LCEL chain
chain = tokenizer_runnable | llm

In [18]:
# Invoke the chain with an example input
response = chain.invoke({"input": "hello"})

# Print the response
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant. Respond naturally to user greetings.<|eot_id|><|start_header_id|>user<|end_header_id|>

hello<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hello! How can I assist you today?


### Not a very readable output ^

In [11]:
from langchain_core.output_parsers import BaseOutputParser
import re

class AssistantResponseParser(BaseOutputParser):
    def parse(self, output: str) -> str:
        # Use regex to extract the assistant's response
        pattern = r"<\|start_header_id\|\>assistant<\|end_header_id\|\>(.*?)(?:<\|eot_id\|\>|$)"
        match = re.search(pattern, output, re.DOTALL)
        if match:
            return match.group(1).strip()
        else:
            return "No assistant response found."

In [12]:
chain = tokenizer_runnable | llm | AssistantResponseParser()

In [13]:
response = chain.invoke({"input": "hello"})
print(response)

Hello! How can I assist you today?


In [14]:
examples = [
    {
        "input": "Can you explain gravity?",
        "output": (
            "## Gravity\n\n"
            "Gravity is one of the fundamental forces in the universe.\n\n"
            "### Discovery\n\n"
            "* Gravity was first discovered by Sir Isaac Newton in the late 17th century.\n"
            "* It was said that Newton theorized about gravity after seeing an apple fall from a tree.\n\n"
            "### In General Relativity\n\n"
            "* Gravity is described as the curvature of spacetime.\n"
            "* The more massive an object is, the more it curves spacetime.\n"
            "* This curvature is what causes objects to fall towards each other.\n\n"
            "### Gravitons\n\n"
            "* Gravitons are hypothetical particles that mediate the force of gravity.\n"
            "* They have not yet been detected.\n\n"
            "**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.\n\n"
        )
    },
    {
        "input": "What is the capital of France?",
        "output": (
            "## France\n\n"
            "The capital of France is Paris.\n\n"
            "### Origins\n\n"
            "* The name Paris comes from the Latin word \"Parisini\" which referred to a Celtic people living in the area.\n"
            "* The Romans named the city Lutetia, which means \"the place where the river turns\".\n"
            "* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.\n\n"
            "**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.\n\n"
        )
    }
]

In [15]:
context = """Aurelio AI is an AI company developing tooling for AI
engineers. Their focus is on language AI with the team having strong
expertise in building AI agents and a strong background in
information retrieval.

The company is behind several open source frameworks, most notably
Semantic Router and Semantic Chunkers. They also have an AI
Platform providing engineers with tooling to help them build with
AI. Finally, the team also provides development services to other
organizations to help them bring their AI tech to market.

Aurelio AI became LangChain Experts in September 2024 after a long
track record of delivering AI solutions built with the LangChain
ecosystem."""

query = "what does Aurelio AI do?"

In [16]:
# Define the system prompt with a placeholder for context
new_system_prompt = """
Answer the user's query based on the context below.
If you cannot answer the question using the provided information answer with "I don't know".

Always answer in markdown format. When doing so please provide headers, short summaries, follow with bullet points, then conclude.

Context: {context}
"""

In [45]:
# Create a custom Runnable to combine context and input
class ContextInputRunnable(Runnable):
    def __init__(self, context):
        self.context = context

    def invoke(self, input_data, config=None):
        # input_data is a dict with 'input' key (e.g., {"input": "What is the speed of light?"})
        formatted_prompt = ChatPromptTemplate.from_messages([
            ("system", new_system_prompt.format(context=self.context)),
            few_shot_prompt,
            ("human", "{input}")
        ])
        return formatted_prompt.format_prompt(input=input_data['input']).to_messages()

In [39]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

In [27]:
from IPython.display import display, Markdown

In [40]:
from langchain.prompts import FewShotChatMessagePromptTemplate

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

out = few_shot_prompt.format()

display(Markdown(out))

Human: Can you explain gravity?
AI: ## Gravity

Gravity is one of the fundamental forces in the universe.

### Discovery

* Gravity was first discovered by Sir Isaac Newton in the late 17th century.
* It was said that Newton theorized about gravity after seeing an apple fall from a tree.

### In General Relativity

* Gravity is described as the curvature of spacetime.
* The more massive an object is, the more it curves spacetime.
* This curvature is what causes objects to fall towards each other.

### Gravitons

* Gravitons are hypothetical particles that mediate the force of gravity.
* They have not yet been detected.

**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.


Human: What is the capital of France?
AI: ## France

The capital of France is Paris.

### Origins

* The name Paris comes from the Latin word "Parisini" which referred to a Celtic people living in the area.
* The Romans named the city Lutetia, which means "the place where the river turns".
* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.

**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.



In [34]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", new_system_prompt),
    few_shot_prompt,
    ("user", "{query}"),
])

In [46]:
# Initialize the custom runnable with the context
context_runnable = ContextInputRunnable(context)

# Create the LCEL chain
chain = tokenizer_runnable | context_runnable | llm | AssistantResponseParser()

# Invoke the chain with an example input
response = chain.invoke({"input":"what does Aurelio AI do?"})
print(response.content)  # Adjust based on your model's output format (e.g., .content for ChatHuggingFace)

TypeError: string indices must be integers, not 'str'